In [1]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from simons_array_python.calibration.planet import ephem_planet
import ephem

INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_runid.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_runid_g3.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_data_quality.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_mapping.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_monitor.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_stim_template.db attached.
INFO: sqlite database /data/pb2/ChileData/databases/pb2a-20211207_benchmark_dataset/pb2a_stim_iq.db attached.
INFO: numpy.fft used.


/opt/pkgs/spt3g_software_sa_2019-05-13/build/spt3g/gcp/ARCExtractor.py:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if uname is not 'None':


In [2]:
def get_error_angle(ra1, dec1, ra2, dec2):
    
    return np.arccos(np.dot(hp.ang2vec(np.pi/2-dec1, ra1), hp.ang2vec(np.pi/2-dec2, ra2)))

## Reproducing your result:

In [3]:
observer = ephem_planet.pb2a_site()
observer.date=45000 #Random date
observer

<ephem.Observer date='2023/3/16 12:00:00' epoch='2000/1/1 12:00:00' lon='-67:47:10.6' lat='-22:57:28.5' elevation=5156.0m horizon=0:00:00.0 temp=15.0C pressure=1010.0mBar>

In [4]:
taua = ephem_planet.source_lookup('taua')
print(taua._ra)
print(taua._dec)
print(taua._epoch)

5:34:31.00
22:01:18.0
2000/1/1 00:00:00


In [5]:
taua.compute(observer)
az, el = taua.az, taua.alt
ra, dec = taua.ra, taua.dec

In [6]:
recon_ra, recon_dec = observer.radec_of(az, el)

In [7]:
print(get_error_angle(ra, dec, recon_ra, recon_dec) * 180/np.pi, 'degrees')

0.32149425218839295 degrees


#### I believe this is consistent with the error you quoted, ~0.3 degrees. The issue is because we're not being consistent with the coordinate system. We need both to either be astrometric, or apparent. In the above code, "Taua_ra" returns apparent coordinates and "radec_of" returns astrometric, which should not be compared

## Repeat of calculation in astrometric coordinates:

In [8]:
observer = ephem_planet.pb2a_site()
observer.date=45000 #Random date
observer.epoch=observer.date #Set the epoch of date
observer

<ephem.Observer date='2023/3/16 12:00:00' epoch='2023/3/16 12:00:00' lon='-67:47:10.6' lat='-22:57:28.5' elevation=5156.0m horizon=0:00:00.0 temp=15.0C pressure=1010.0mBar>

In [9]:
taua.compute(observer)
az, el = taua.az, taua.alt
ra, dec = taua.a_ra, taua.a_dec #Since radec_of returns astrometric coordinates, we should compare with astrometric instead of apparent

In [10]:
recon_ra, recon_dec = observer.radec_of(az, el)

In [11]:
print(get_error_angle(ra, dec, recon_ra, recon_dec) * 180/np.pi * 60 * 60, 'arcseconds')

0.011903944008107542 arcseconds


## Repeat of calculation in apparent coordinates:

In [12]:
observer = ephem_planet.pb2a_site()
observer.date=45000 #Random date
observer.epoch=-9786 #Returns apparent ra/dec
observer

<ephem.Observer date='2023/3/16 12:00:00' epoch='1873/3/16 12:00:00' lon='-67:47:10.6' lat='-22:57:28.5' elevation=5156.0m horizon=0:00:00.0 temp=15.0C pressure=1010.0mBar>

In [13]:
taua.compute(observer)
az, el = taua.az, taua.alt
ra, dec = taua.ra, taua.dec #Apparent ra/dec

In [14]:
recon_ra, recon_dec = observer.radec_of(az, el)

In [15]:
print(get_error_angle(ra, dec, recon_ra, recon_dec) * 180/np.pi * 60 * 60, 'arcseconds')

0.011903944008107542 arcseconds
